In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

TensorFlow version: 2.6.0
Num GPUs Available:  1


In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

2021-11-19 22:25:55.850633: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Internal: Couldn't parse JSON response from OAuth server.".


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/parsayousefi/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Metal device set to: Apple M1 Max


2021-11-19 22:25:59.102718: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-19 22:25:59.103095: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

In [5]:
batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                           activation='relu'),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                           activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    # tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[
        'acc', 'mae'
    ],
)

In [8]:
with tf.device('/device:GPU:0'):
    model.fit(
        ds_train,
        epochs=20,
        validation_data=ds_test,
    )

Epoch 1/20


2021-11-19 22:26:23.982443: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-19 22:26:23.983487: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-19 22:26:23.983565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


466/469 [============================>.] - ETA: 0s - loss: 0.1691 - acc: 0.9508 - mae: 4.3728

2021-11-19 22:26:33.298507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 10s 13ms/step - loss: 0.1686 - acc: 0.9510 - mae: 4.3737 - val_loss: 0.0648 - val_acc: 0.9783 - val_mae: 4.3630
Epoch 2/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0458 - acc: 0.9863 - mae: 4.3737 - val_loss: 0.0360 - val_acc: 0.9876 - val_mae: 4.3630
Epoch 3/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0282 - acc: 0.9913 - mae: 4.3737 - val_loss: 0.0433 - val_acc: 0.9854 - val_mae: 4.3630
Epoch 4/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0211 - acc: 0.9933 - mae: 4.3737 - val_loss: 0.0316 - val_acc: 0.9898 - val_mae: 4.3630
Epoch 5/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0139 - acc: 0.9957 - mae: 4.3737 - val_loss: 0.0298 - val_acc: 0.9911 - val_mae: 4.3630
Epoch 6/20
469/469 [==============================] - 6s 12ms/step - loss: 0.0092 - acc: 0.9973 - mae: 4.3737 - val_loss: 0.0379 - val_acc: 0.9887 - val_mae: 4.3630
Epoch 7/20
469/469 [